In [ ]:
import streamlit as st
import pandas as pd
import urllib
import json
from helpers import *

2026-02-06 18:42:08.556 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [4]:
station_url = "https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_status.json"
location_url = "https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_information"

One_url = "https://data.smartdublin.ie/dublinbikes-api/bikes/dublin_bikes/current/stations.geojson"

In [8]:
def query_station_status(url):
    with urllib.request.urlopen(url) as data_url:  # Open the URL
        data = json.loads(data_url.read().decode())  # Read and decode the JSON data
    df = pd.DataFrame(data["data"]["stations"])  # Convert the data to a DataFrame
    df = df[df.is_renting == 1]  # Filter out stations that are not renting
    df = df[df.is_returning == 1]  # Filter out stations that are not returning
    df = df.drop_duplicates(["station_id", "last_reported"])  # Remove duplicate records
    df.last_reported = df.last_reported.map(
        lambda x: dt.datetime.utcfromtimestamp(x)
    )  # Convert timestamps to datetime
    df["time"] = data["last_updated"]  # Add the last updated time to the DataFrame
    df.time = df.time.map(
        lambda x: dt.datetime.utcfromtimestamp(x)
    )  # Convert timestamps to datetime
    df = df.set_index("time")  # Set the time as the index
    df.index = df.index.tz_localize("UTC")  # Localize the index to UTC
    df = pd.concat(
        [df, df["num_bikes_available_types"].apply(pd.Series)], axis=1
    )  # Expand the bike types column

    return df  # Return the DataFrame




In [ ]:
import pandas as pd
import urllib.request
import json
import datetime as dt

def query_station_status_geojson(url):
    with urllib.request.urlopen(url) as data_url:
        data = json.loads(data_url.read().decode())

    records = []

    for feature in data["features"]:
        props = feature["properties"].copy()
        lon, lat = feature["geometry"]["coordinates"]

        props["lon"] = lon
        props["lat"] = lat

        records.append(props)

    df = pd.DataFrame(records)

    # Filters (same as your original)
    df = df[df.is_renting == 1]
    df = df[df.is_returning == 1]

    df = df.drop_duplicates(["station_id", "last_reported"])

    # Convert timestamps
    df["last_reported"] = df["last_reported"].apply(
        lambda x: dt.datetime.utcfromtimestamp(x)
    )

    # Use last_reported as time index
    df = df.set_index("last_reported")
    df.index = df.index.tz_localize("UTC")

    # Expand bike types
    if "num_bikes_available_types" in df.columns:
        df = pd.concat(
            [df, df["num_bikes_available_types"].apply(pd.Series)],
            axis=1
        )

    return df


In [17]:
query_station_status_geojson(One_url)

C:\Users\piyus\AppData\Local\Temp\ipykernel_3192\527795415.py:31: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  lambda x: dt.datetime.utcfromtimestamp(x)


,system_id,station_id,num_bikes_available,num_docks_available,is_installed,is_renting,is_returning,last_updated,name,short_name,address,region_id,capacity,last_reported_dt,longitude,latitude
last_reported,,,,,,,,,,,,,,,,
2026-02-07 00:16:40+00:00,dublin_bikes,1,0,31,True,True,True,1770423400,CLARENDON ROW,,Clarendon Row,,31,2026-02-07 00:16:40,-6.262501,53.340927
2026-02-07 00:17:34+00:00,dublin_bikes,10,10,6,True,True,True,1770423454,DAME STREET,,Dame Street,,16,2026-02-07 00:17:34,-6.266802,53.344006
2026-02-07 00:09:41+00:00,dublin_bikes,100,25,0,True,True,True,1770422981,HEUSTON BRIDGE (SOUTH),,Heuston Bridge (South),,25,2026-02-07 00:09:41,-6.292041,53.347107
2026-02-07 00:16:53+00:00,dublin_bikes,101,14,16,True,True,True,1770423413,KING STREET NORTH,,King Street North,,30,2026-02-07 00:16:53,-6.273507,53.350292
2026-02-07 00:14:29+00:00,dublin_bikes,102,27,13,True,True,True,1770423269,WESTERN WAY,,Western Way,,40,2026-02-07 00:14:29,-6.269425,53.354931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-02-07 00:15:44+00:00,dublin_bikes,95,39,1,True,True,True,1770423344,ROYAL HOSPITAL,,Royal Hospital,,40,2026-02-07 00:15:44,-6.297060,53.343899
2026-02-07 00:13:27+00:00,dublin_bikes,96,8,22,True,True,True,1770423207,KILMAINHAM LANE,,Kilmainham Lane,,30,2026-02-07 00:13:27,-6.305085,53.341805
2026-02-07 00:12:34+00:00,dublin_bikes,97,29,11,True,True,True,1770423154,KILMAINHAM GAOL,,Kilmainham Gaol,,40,2026-02-07 00:12:34,-6.310015,53.342113


In [18]:
import pandas as pd
import urllib.request
import json

def get_station_latlon_geojson(url):
    with urllib.request.urlopen(url) as data_url:
        data = json.loads(data_url.read().decode())

    records = []

    for feature in data["features"]:
        props = feature["properties"]
        lon, lat = feature["geometry"]["coordinates"]

        records.append({
            "station_id": props["station_id"],
            "name": props.get("name"),
            "latitude": lat,
            "longitude": lon
        })

    return pd.DataFrame(records)


In [19]:
get_station_latlon_geojson(One_url)

,station_id,name,latitude,longitude
0,1,CLARENDON ROW,53.340927,-6.262501
1,10,DAME STREET,53.344006,-6.266802
2,100,HEUSTON BRIDGE (SOUTH),53.347107,-6.292041
3,101,KING STREET NORTH,53.350292,-6.273507
4,102,WESTERN WAY,53.354931,-6.269425
...,...,...,...,...
109,95,ROYAL HOSPITAL,53.343899,-6.297060
110,96,KILMAINHAM LANE,53.341805,-6.305085
111,97,KILMAINHAM GAOL,53.342113,-6.310015
112,98,FREDERICK STREET SOUTH,53.341515,-6.256853
